In [ ]:
##########################
##########################
###### SECTION 4 #########
##########################
##########################

#########################################
# Data Preparation for Machine Learning #
#########################################

import pandas as pd
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.inspection import permutation_importance



#Filter out sex, age, income and comorbidities 
columns_of_interest = [
    'slider_sex', 'age_group', 'income', 'comorbid_aids_hiv', 'comorbid_asthma',
    'comorbid_chronic_cardiac_disease', 'comorbid_chronic_haematological_disease',
    'comorbid_chronic_kidney_disease', 'comorbid_chronic_neurological_disorder',
    'comorbid_chronic_pulmonary_disease', 'comorbid_dementia', 'comorbid_diabetes',
    'comorbid_hypertension', 'comorbid_immunosuppression', 'comorbid_liver_disease',
    'comorbid_malignant_neoplasm', 'comorbid_malnutrition', 'comorbid_obesity',
    'comorbid_other', 'comorbid_rare_diseases_and_inborn_errors_of_metabolism',
    'comorbid_rheumatologic_disorder', 'comorbid_smoking', 'comorbid_transplantation',
    'comorbid_tuberculosis', 'comorbid_pregnancy', 'MACE'
]

data = data[columns_of_interest].copy()

# Target column
target_col = 'MACE'

data = data.astype(str)

# Separate features and target
X = data.drop(columns=[target_col])
y = data[target_col]

# Separate ordinal and nominal features
ordinal_features = ['age_group', 'income']
nominal_features = [col for col in X.columns if col not in ordinal_features]

# Define the order for ordinal features
ordinal_encoder = OrdinalEncoder(categories=[age_labels, 
                                             ['Low income', 
                                              'Lower middle income', 
                                              'Upper middle income',
                                              'High income']])

# Use ordinal encoding for ordinal features
X[ordinal_features] = ordinal_encoder.fit_transform(X[ordinal_features])

# Use one hot encoding for nominal features
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_nominal_data = encoder.fit_transform(X[nominal_features])

# Combine ordinal and encoded nominal data
combined_data = np.hstack([X[ordinal_features].values, encoded_nominal_data])

# Encode the target variable
y = LabelEncoder().fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_data, y, 
                                                    test_size=0.3, random_state=42)


#################################
# Linear Discriminant Analysis #
################################


# Initialize LDA classifier
lda_clf = LinearDiscriminantAnalysis()

# Define parameter grid
param_grid_lda = {
    "solver": ['svd', 'lsqr', 'eigen'],  
    "shrinkage": [None, 'auto',0.1, 0.2, 0.3, 0.4 ,0.5, 1]  
}

# Perform grid search with cross-validation 
grid_search_lda = GridSearchCV(lda_clf, param_grid_lda, n_jobs=-1, verbose=2, 
                               cv=5, return_train_score=True)

# Fit the grid search to training data
grid_search_lda.fit(X_train, y_train)

# Print the best parameters
print("Best parameters for LDA:", grid_search_lda.best_params_)

# Evaluate the LDA classifier
best_lda = grid_search_lda.best_estimator_
accuracy_test_lda = best_lda.score(X_test, y_test)

print('LDA - Accuracy on Test Set: ', accuracy_test_lda)

# Convert cv_results_ to a DataFrame
cv_results_lda = pd.DataFrame(grid_search_lda.cv_results_)
cv_results_lda = cv_results_lda.sort_values(by = 'mean_test_score',ascending = False)

# Print the DataFrame to see mean training and mean validation accuracies
print(cv_results_lda)


# AUC calculation is shown for LDA
# All other machine learning methods use same way



# Predict probabilities for positive class 
y_pred_proba = best_lda.predict_proba(X_test)[:, 1]

# Calculate the AUC score
auc_score_lda = roc_auc_score(y_test, y_pred_proba)

print('AUC Score: ', auc_score_lda)

# Number of bootstrap samples
n_bootstraps = 1000
rng = np.random.RandomState(42)
bootstrapped_aucs = []

# Perform bootstrapping
for i in range(n_bootstraps):
    X_resampled, y_resampled = resample(X_test, y_test, random_state=rng)
    y_pred_proba = best_lda.predict_proba(X_resampled)[:, 1]
    
    # Calculate AUC
    if len(np.unique(y_resampled)) < 2:
        continue
    auc = roc_auc_score(y_resampled, y_pred_proba)
    bootstrapped_aucs.append(auc)

# Calculate 95% confidence interval
alpha = 0.95
lower = np.percentile(bootstrapped_aucs, (1 - alpha) * 100 / 2)
upper = np.percentile(bootstrapped_aucs, (alpha + (1 - alpha) / 2) * 100)

print(f'95% confidence interval for the AUC: [{lower:.4f} - {upper:.4f}]')




###################################
# Quadratic Discriminant Analysis #
###################################



# Initialize QDA classifier
qda_clf = QuadraticDiscriminantAnalysis()

# Define parameter grid
param_grid_qda = {
    "reg_param": [0.1, 0.2, 0.3, 0.35, 0.4, 0.6, 0.8, 1.0]
}

# Perform grid search with cross-validation
grid_search_qda = GridSearchCV(qda_clf, param_grid_qda, n_jobs=-1, verbose=2,
                               cv=5, return_train_score=True)

# Fit grid search to the training data
grid_search_qda.fit(X_train, y_train)


###########################
# Multi-Layer Perceptron #
##########################


# Initialize MLP classifier
mlp_clf = MLPClassifier(max_iter=1000, random_state=42)

# Define parameter grid
param_grid = {
    "hidden_layer_sizes": [(50,), (100,), (150,), (50, 100), (100, 50)], 
    "activation": ['tanh', 'relu'],  
    "solver": ['sgd', 'adam'],
    "learning_rate": ['adaptive', 'constant']  
}

# Perform grid search with cross-validation
grid_search_mlp = GridSearchCV(mlp_clf, param_grid, n_jobs=-1, verbose=2, 
                               cv=5, return_train_score=True)

# Fit grid search tO training data
grid_search_mlp.fit(X_train, y_train)



################
# Naive Bayes #
################



# Define Naive Bayes classifier
nb_clf = CategoricalNB()

# Parameter grid for GridSearchCV
param_grid = {
    "fit_prior": [True, False]
}

# Perform grid search with cross-validation
grid_search_nb = GridSearchCV(nb_clf, param_grid, n_jobs=-1, verbose=2, 
                              cv=5, return_train_score=True)

# Fit grid search to training data
grid_search_nb.fit(X_train, y_train)


#################
# Random Forest #
#################


# Initialize the Random Forest classifier
rf_clf = RandomForestClassifier(random_state=42)

# Define parameter grid
param_grid = {
    "n_estimators": [1, 2, 3, 4, 5, 10, 50],  
    "max_depth": [2, 3, 10, 20],  
    "min_samples_split": [2, 3, 5, 10],  
    "min_samples_leaf": [2, 3, 4],  
    "bootstrap": [True, False]  
}


# Perform grid search with cross-validation
grid_search_rf = GridSearchCV(rf_clf, param_grid, n_jobs=-1, verbose=2, 
                              cv=5, return_train_score=True)

# Fit grid search to training data
grid_search_rf.fit(X_train, y_train)
best_rf = grid_search_rf.best_estimator_

# Get the feature importances
feature_importances_rf = best_rf.feature_importances_

# Combine the feature names from ordinal and one-hot encoded features
feature_names = ordinal_features + list(encoder.get_feature_names_out(nominal_features))

# Create a DataFrame for the scores
feature_data_rf = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances_rf
})

# Sort by importance scores in descending order
feature_data_rf = feature_data_rf.sort_values(by='Importance',
                                              ascending=False)
print(feature_data_rf)



############
# AdaBoost #
############


# Define AdaBoost classifier
ada_clf = AdaBoostClassifier()

# Parameter grid for GridSearchCV
param_grid = {
    "n_estimators": [50, 100, 200, 300, 350, 400, 450],          
    "learning_rate": [0.01, 0.04, 0.05, 0.06, 0.1, 0.5, 1.0],   
}

# Perform grid search with cross-validation
grid_search_ada = GridSearchCV(ada_clf, param_grid, n_jobs=-1, verbose=2, 
                               cv=5, return_train_score=True)

# Fit grid search to training data
grid_search_ada.fit(X_train, y_train)
best_ada = grid_search_ada.best_estimator_

# Get the feature importances
feature_importances_ada = best_ada.feature_importances_

# Create a DataFrame for the scores
feature_data_ada = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances_ada
})

# Sort by importance scores in descending order
feature_data_ada = feature_data_ada.sort_values(by='Importance',
                                              ascending=False)
print(feature_data_ada)


############
# XGBoost #
############


# Define the XGBoost classifier
xgb_clf = XGBClassifier(use_label_encoder=False)

# Parameter grid for GridSearchCV
param_grid = {
    "learning_rate": [0.1, 0.2, 0.3, 0.4, 0.5],  
    "n_estimators": [100, 200, 250, 300, 500],  
    "max_depth": [2, 3, 4, 5],            
    "reg_alpha": [0.1, 0.2, 0.3],      
    "reg_lambda": [10, 50, 85, 90, 95]         
}

# Perform grid search with cross-validation
grid_search_xgb = GridSearchCV(xgb_clf, param_grid, n_jobs=-1, verbose=2, 
                               cv=5, return_train_score=True)

# Fit grid search to training data
grid_search_xgb.fit(X_train, y_train)
best_xgb = grid_search_xgb.best_estimator_

# Get the feature importances
feature_importances_xgb = best_xgb.feature_importances_

# Create a DataFrame for the scores
feature_data_xgb = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances_xgb
})

# Sort by importance scores in descending order
feature_data_xgb = feature_data_xgb.sort_values(by='Importance',
                                              ascending=False)
print(feature_data_xgb)



###############################
# Histogram Gradient Boosting #
###############################



# Define the HistGradientBoosting classifier
hist_gb_clf = HistGradientBoostingClassifier()

# Parameter grid for GridSearchCV
param_grid = {
    "learning_rate": [0.01, 0.05, 0.09, 0.1, 0.11], 
    "max_iter": [100, 200, 250, 300, 500],        
    "max_leaf_nodes": [30, 40, 45, 50, 60],      
    "max_depth": [None, 5, 10],       
    "min_samples_leaf": [5, 8, 10, 12, 20],    
    "l2_regularization": [0.0, 0.01, 0.1]  
}

# Perform grid search with cross-validation
grid_search_hist_gb = GridSearchCV(hist_gb_clf, param_grid, n_jobs=-1, verbose=2,
                                   cv=5, return_train_score=True)

# Fit grid search to training data
grid_search_hist_gb.fit(X_train, y_train)
best_hist_gb = grid_search_hist_gb.best_estimator_


# Perform permutation importance
result = permutation_importance(best_hist_gb, X_test, y_test, 
                                n_repeats=10, random_state=42, n_jobs=-1)

# Get feature names after encoding
ordinal_features = ['age_group', 'income']
one_hot_feature_names = encoder.get_feature_names_out(nominal_features)  
feature_names = np.concatenate([ordinal_features, one_hot_feature_names])

# Sort the permutation importances
sorted_importances_idx = result.importances_mean.argsort()[::-1] 
importances = result.importances_mean[sorted_importances_idx]
sorted_feature_names = feature_names[sorted_importances_idx]

# Print the sorted feature names along with their importances
print(sorted_feature_names)
print(importances)

